# DataSet Preparation Utils
by Wael Radwan

#### Library used to prepare datasets from data
continans various tools to prepare the training dataset and testing dataset

mainly used to randomize files of the dataset
divide the dataset into traininig and testing dataset
randomize the test files

<b> the main idea here is to load the dataset into classified directories then use this utility to create train/test dataset with the randomization </b>

In [1]:
import os
import re
import random
import ntpath
import shutil
import pandas as pd
import numpy as np

#### Input description
input to this tool is a dataset of files divided into classes in the following order
- class_1:
    - file_1
    - file_2
    - .
    - .
- class_2
    - file_1
    - file_2
    - .
    - .
- .
- .


- class_n 
    - file_1
    - file_2
    - .
    - .


#### Output description
output shoud be the data divided into train and test sets  in the following order

- train
    - class_1_001
    - class_1_002
    - .
    - .
    - class_2_001
    - class_2_002
    - .
    - .
    - .
    - .
    - class_n_001
    - class_n_002
    - .
    - .
 
- test
    - class_1_001
    - class_1_002
    - .
    - .
    - class_2_001
    - class_2_002
    - .
    - .
    - .
    - .
    - class_n_001
    - class_n_002
    - .
    - .
 
    - class_label.csv (a csv file contains the filename and the label so it can be used for testing comparison)

#### create panda from the files dataset
mainly used for the test dataset to generate the file name and the label based on the label criteria
randomize is set to true if the files are ranomized in the dataset instead of ordered

In [2]:
def list_files_with_labels(path, randomize=False):
    dataset = []
    files = sorted(os.listdir(path))
    if not files:
        return dataset
    file = [os.path.splitext(x)[0] for x in files]
    ext = [os.path.splitext(x)[-1] for x in files]
    class_name = ntpath.basename(path)
    

    # create panda dataframe for file sorted by their name

    dataset = pd.DataFrame(data={"file":file,"ext":ext, "label":class_name})
    dataset = dataset[['file', 'ext', 'label']]

    # if randomize is set to True then shuffle the dataset dataframe
    if (randomize) :
        dataset= dataset.sample(frac=1)
        dataset = dataset.sample(frac=1).reset_index(drop=True)

    return dataset


#### create list of the dataset with new name for each file based on a sequence
randomize or order the files in the dataset then create a new name based on sequence number

- path      : path to the class
- seqlen    : is the lenght of the sequence so if seqlen = 3 the new file names follow the sequence 001, 002, 003, ...
- randomize : either to randomize the files in the class set or not (default is not to randomize) 
- retrun panda structure with new file name 

In [20]:
def list_with_new_file_name_seq(path,  seqlen, randomize=False,):
    df = []  
    dataset = list_files_with_labels( path , randomize)
    #if not dataset:
     #   return
    
    # recreate index in case the dataset has been randomize
    # so the seq_number will be reflected in out file_name
    df = dataset.sample(frac=1).reset_index(drop=True)
    
    # new file name will be classid_seq(of seq_len).ext
    # for example 
    # 1_0001.mp3
    # 1_0002.mp3
    df['newfilename'] = df['label']+'_'+(df.index + 1).astype(str).str.zfill(seqlen) + df.ext
    
    # df.index = df.index.map(str) 
    return df

In [21]:
path='/ds/dataset/quran/999'
dataset = list_files_with_labels( path,randomize=True)
data = c

In [22]:
data

,file,ext,label,newfilename
0,024,.mp3,999,999_001.mp3
1,031,.mp3,999,999_002.mp3
2,029,.mp3,999,999_003.mp3
3,015,.mp3,999,999_004.mp3
4,012,.mp3,999,999_005.mp3
5,026,.mp3,999,999_006.mp3
6,032,.mp3,999,999_007.mp3
7,005,.mp3,999,999_008.mp3
8,013,.mp3,999,999_009.mp3
9,022,.mp3,999,999_010.mp3


#### Rename files 
After creating the dataframe with proposed renamed files then do the rename files in the dataset
then rename all the files to _tmp files in order not to overwrite files that already exist 


In [35]:
def rename_files_dataset(path, seqlen):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_with_new_file_name_seq( path,seqlen)
    #if not data:
     #   return
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        tmp_file_name = path+'/'+row['newfilename'] + '_tmp'
        orig_file_name =  path+'/'+row['file'] + row['ext']
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        os.rename(orig_file_name, tmp_file_name)
        tmp_file_names.append(tmp_file_name)
        
    # now remove _tmp files    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
        os.rename(temp_file, temp_file.rsplit('_tmp', -1)[0]) 
    return

##### splitt the dataset into training and test
here split the dataset into training and testing 
- path : path to the class
- train_path : path to the train dir
- test_path : path to the test dir
- train_files_count : is the number of files in each class to be moved to the training dir
- randomize : either to randomize the files in the class set or not (default is not to randomize)

the output will be copying the data into train and test directories

In [49]:
def split_into_train_test( path, train_path, test_path,train_files_count, randomize=True):
    # read the files in the origianl path ordered by there name
    data = list_files_with_labels( path,randomize)
    # copy the dir name of the class 
    # this only copy the current directory name (without the path)
    dir_name = ntpath.basename(path)
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        orig_file_name =  path+'/'+row['file'] + row['ext']
        if (index<train_files_count):
            tmp_file_name = train_path+'/'+row['file']  + row['ext']
        else :
            tmp_file_name = test_path+'/'+row['file']  + row['ext']
       
        print('copy %s to %s'  % (orig_file_name, tmp_file_name) )
        shutil.copy(orig_file_name, tmp_file_name)
    return

In [50]:
train_path='/ds/dataset/quran/train'
test_path='/ds/dataset/quran/test'
split_into_train_test( path, train_path, test_path,10)

copy /ds/dataset/quran/999/999_0032.mp3 to /ds/dataset/quran/train/999_0032.mp3
copy /ds/dataset/quran/999/999_0024.mp3 to /ds/dataset/quran/train/999_0024.mp3
copy /ds/dataset/quran/999/999_0025.mp3 to /ds/dataset/quran/train/999_0025.mp3
copy /ds/dataset/quran/999/999_0004.mp3 to /ds/dataset/quran/train/999_0004.mp3
copy /ds/dataset/quran/999/999_0003.mp3 to /ds/dataset/quran/train/999_0003.mp3
copy /ds/dataset/quran/999/999_0030.mp3 to /ds/dataset/quran/train/999_0030.mp3
copy /ds/dataset/quran/999/999_0005.mp3 to /ds/dataset/quran/train/999_0005.mp3
copy /ds/dataset/quran/999/999_0021.mp3 to /ds/dataset/quran/train/999_0021.mp3
copy /ds/dataset/quran/999/999_0020.mp3 to /ds/dataset/quran/train/999_0020.mp3
copy /ds/dataset/quran/999/999_0017.mp3 to /ds/dataset/quran/train/999_0017.mp3
copy /ds/dataset/quran/999/999_0002.mp3 to /ds/dataset/quran/test/999_0002.mp3
copy /ds/dataset/quran/999/999_0022.mp3 to /ds/dataset/quran/test/999_0022.mp3
copy /ds/dataset/quran/999/999_0011.mp3 to

##### shuffle  dataset  files

shuffle files in the testing dataset then rename all the files to _tmp files in order not to overwrite files that already exist

** this can be used to shuffle any directory but the main reason here is to make sure that the testing dataset is shuffled in order to avoid overfitting

In [229]:
def shuffle_dataset(path):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_files_with_labels( path,randomize=True)
    # create a new column with the random file selected from the same dataset
    # so the files will be shuffled
    data['newfile'] = data["file"].transform(np.random.permutation)


     
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in data.iterrows():
        tmp_file_name = path+'/'+row['newfile'] + row['ext'] + '_tmp'
        orig_file_name =  path+'/'+row['file'] + row['ext']
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        os.rename(orig_file_name, tmp_file_name)
        tmp_file_names.append(tmp_file_name)
        
    # now remove _tmp f    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
        os.rename(temp_file, temp_file.rsplit('_tmp', -1)[0])
    return

###### to be continued
from here we will continue by adding more features
-  generate a csv description file for test to be used kaggle 

##### Generate the files / lables csv file
finally generate the csv file that holds the correct mapping of the files and thier correct labels this file should be used as a benchmark against files submitted

In [103]:
def generate_test_csv(test_path, csv_file_name):
    test_dataset = list_files_with_labels( test_path,randomize=True)
    test_dataset['fullfilename'] = test_dataset['file'] + test_dataset['ext']
    header = ["fullfilename", "label"]
    test_dataset.to_csv( csv_file_name, index=False, quoting=3 ,columns = header)
    return

#### Example code
the following sample code could be used to generate train / test dataset from the dataset

-- you can copy it and paste it in the code 

In [71]:
# first of all define the location of the files
print('----Start----')
path='/ds/dataset/quran/112'
train_path='/ds/dataset/quran/train'
test_path='/ds/dataset/quran/test'
csv_file_name = '/ds/dataset/quran/test_dataset.csv'

#first rename files in the path to sequenced files prefixed with label 
print('rename file in dataset %s' % (path))

rename_files_dataset(path, 4)

# suffle the files in the path
print('shuffle files in dataset %s' % (path))
shuffle_dataset(path)

# then split into train and test
print('then split into train and test')
split_into_train_test( path, train_path, test_path,20)

# suffle the files in the test
print('shuffle test dataset')
shuffle_test_dataset(test_path)

# write csv file
csv_file_name ='test_dataset.csv'
csv_file= path+'/'+csv_file_name
generate_test_csv(test_path, csv_file)

----Start----
rename file in dataset /ds/dataset/quran/112
rename /ds/dataset/quran/112/112_0001.mp3 to /ds/dataset/quran/112/112_0001.mp3_tmp
rename /ds/dataset/quran/112/112_0224.mp3 to /ds/dataset/quran/112/112_0002.mp3_tmp
rename /ds/dataset/quran/112/112_0284.mp3 to /ds/dataset/quran/112/112_0003.mp3_tmp
rename /ds/dataset/quran/112/112_0378.mp3 to /ds/dataset/quran/112/112_0004.mp3_tmp
rename /ds/dataset/quran/112/112_0203.mp3 to /ds/dataset/quran/112/112_0005.mp3_tmp
rename /ds/dataset/quran/112/112_0323.mp3 to /ds/dataset/quran/112/112_0006.mp3_tmp
rename /ds/dataset/quran/112/112_0210.mp3 to /ds/dataset/quran/112/112_0007.mp3_tmp
rename /ds/dataset/quran/112/112_0019.mp3 to /ds/dataset/quran/112/112_0008.mp3_tmp
rename /ds/dataset/quran/112/112_0449.mp3 to /ds/dataset/quran/112/112_0009.mp3_tmp
rename /ds/dataset/quran/112/112_0415.mp3 to /ds/dataset/quran/112/112_0010.mp3_tmp
rename /ds/dataset/quran/112/112_0178.mp3 to /ds/dataset/quran/112/112_0011.mp3_tmp
rename /ds/datase

rename /ds/dataset/quran/112/112_0310.mp3 to /ds/dataset/quran/112/112_0390.mp3_tmp
rename /ds/dataset/quran/112/112_0019.mp3 to /ds/dataset/quran/112/112_0384.mp3_tmp
rename /ds/dataset/quran/112/112_0174.mp3 to /ds/dataset/quran/112/112_0342.mp3_tmp
rename /ds/dataset/quran/112/112_0217.mp3 to /ds/dataset/quran/112/112_0406.mp3_tmp
rename /ds/dataset/quran/112/112_0150.mp3 to /ds/dataset/quran/112/112_0196.mp3_tmp
rename /ds/dataset/quran/112/112_0418.mp3 to /ds/dataset/quran/112/112_0296.mp3_tmp
rename /ds/dataset/quran/112/112_0149.mp3 to /ds/dataset/quran/112/112_0291.mp3_tmp
rename /ds/dataset/quran/112/112_0233.mp3 to /ds/dataset/quran/112/112_0427.mp3_tmp
rename /ds/dataset/quran/112/112_0223.mp3 to /ds/dataset/quran/112/112_0432.mp3_tmp
rename /ds/dataset/quran/112/112_0090.mp3 to /ds/dataset/quran/112/112_0290.mp3_tmp
rename /ds/dataset/quran/112/112_0341.mp3 to /ds/dataset/quran/112/112_0050.mp3_tmp
rename /ds/dataset/quran/112/112_0086.mp3 to /ds/dataset/quran/112/112_0443.

copy /ds/dataset/quran/112/112_0220.mp3 to /ds/dataset/quran/test/112_0220.mp3
copy /ds/dataset/quran/112/112_0014.mp3 to /ds/dataset/quran/test/112_0014.mp3
copy /ds/dataset/quran/112/112_0137.mp3 to /ds/dataset/quran/test/112_0137.mp3
copy /ds/dataset/quran/112/112_0041.mp3 to /ds/dataset/quran/test/112_0041.mp3
copy /ds/dataset/quran/112/112_0423.mp3 to /ds/dataset/quran/test/112_0423.mp3
copy /ds/dataset/quran/112/112_0320.mp3 to /ds/dataset/quran/test/112_0320.mp3
copy /ds/dataset/quran/112/112_0018.mp3 to /ds/dataset/quran/test/112_0018.mp3
copy /ds/dataset/quran/112/112_0379.mp3 to /ds/dataset/quran/test/112_0379.mp3
copy /ds/dataset/quran/112/112_0242.mp3 to /ds/dataset/quran/test/112_0242.mp3
copy /ds/dataset/quran/112/112_0154.mp3 to /ds/dataset/quran/test/112_0154.mp3
copy /ds/dataset/quran/112/112_0102.mp3 to /ds/dataset/quran/test/112_0102.mp3
copy /ds/dataset/quran/112/112_0268.mp3 to /ds/dataset/quran/test/112_0268.mp3
copy /ds/dataset/quran/112/112_0238.mp3 to /ds/datas

copy /ds/dataset/quran/112/112_0058.mp3 to /ds/dataset/quran/test/112_0058.mp3
copy /ds/dataset/quran/112/112_0363.mp3 to /ds/dataset/quran/test/112_0363.mp3
copy /ds/dataset/quran/112/112_0219.mp3 to /ds/dataset/quran/test/112_0219.mp3
copy /ds/dataset/quran/112/112_0317.mp3 to /ds/dataset/quran/test/112_0317.mp3
copy /ds/dataset/quran/112/112_0098.mp3 to /ds/dataset/quran/test/112_0098.mp3
copy /ds/dataset/quran/112/112_0138.mp3 to /ds/dataset/quran/test/112_0138.mp3
copy /ds/dataset/quran/112/112_0225.mp3 to /ds/dataset/quran/test/112_0225.mp3
copy /ds/dataset/quran/112/112_0346.mp3 to /ds/dataset/quran/test/112_0346.mp3
copy /ds/dataset/quran/112/112_0122.mp3 to /ds/dataset/quran/test/112_0122.mp3
copy /ds/dataset/quran/112/112_0310.mp3 to /ds/dataset/quran/test/112_0310.mp3
copy /ds/dataset/quran/112/112_0339.mp3 to /ds/dataset/quran/test/112_0339.mp3
copy /ds/dataset/quran/112/112_0053.mp3 to /ds/dataset/quran/test/112_0053.mp3
copy /ds/dataset/quran/112/112_0113.mp3 to /ds/datas

rename /ds/dataset/quran/test/112_0070.mp3 to /ds/dataset/quran/test/112_0320.mp3_tmp
rename /ds/dataset/quran/test/112_0059.mp3 to /ds/dataset/quran/test/112_0455.mp3_tmp
rename /ds/dataset/quran/test/112_0330.mp3 to /ds/dataset/quran/test/112_0104.mp3_tmp
rename /ds/dataset/quran/test/112_0223.mp3 to /ds/dataset/quran/test/112_0073.mp3_tmp
rename /ds/dataset/quran/test/112_0189.mp3 to /ds/dataset/quran/test/112_0076.mp3_tmp
rename /ds/dataset/quran/test/112_0222.mp3 to /ds/dataset/quran/test/112_0094.mp3_tmp
rename /ds/dataset/quran/test/112_0209.mp3 to /ds/dataset/quran/test/112_0173.mp3_tmp
rename /ds/dataset/quran/test/112_0310.mp3 to /ds/dataset/quran/test/112_0241.mp3_tmp
rename /ds/dataset/quran/test/112_0033.mp3 to /ds/dataset/quran/test/112_0154.mp3_tmp
rename /ds/dataset/quran/test/112_0424.mp3 to /ds/dataset/quran/test/112_0139.mp3_tmp
rename /ds/dataset/quran/test/112_0169.mp3 to /ds/dataset/quran/test/112_0322.mp3_tmp
rename /ds/dataset/quran/test/112_0121.mp3 to /ds/data

#### create panda from test files dataset
mainly used for the test dataset to generate the file name and the label based on the label criteria randomize is set to true if the files are ranomized in the dataset instead of ordered
will return the panda with the original file name and the new file name 

In [168]:
def list_files_with_xlabels(path, randomize=False):
    dataset = []
    files = sorted(os.listdir(path))
    np.array(file).reshape(len(file),3)
    dataset = pd.DataFrame(np.array(file).reshape(len(file),3), columns =  ['label', 'file', 'ext'] )

    # if randomize is set to True then shuffle the dataset dataframe
    if (randomize) :
        dataset= dataset.sample(frac=1)
        dataset = dataset.sample(frac=1).reset_index(drop=True)

    return dataset

#### shuffle  test dataset  files

shuffle files in the <b>testing dataset </b> then rename all the files to _tmp files in order not to overwrite files that already exist

** this can be used to shuffle any directory but the main reason here is to make sure that the testing dataset is shuffled in order to avoid overfitting

In [230]:
def shuffle_files_test_dataset(path):
    # first list the files in the input dir and put the porposed name in the dataframe
    data = list_files_with_xlabels( path,randomize=True)

    # recreate index in case the dataset has been randomize
    # so the seq_number will be reflected in out file_name
    df = data.sample(frac=1).reset_index(drop=True)
    
    # new file name will be classid_seq(of seq_len).ext
    # for example 
    # 1_0001.mp3
    # 1_0002.mp3
    
    # find the maximum number of files then use this as a sequence length
    seqlen= len(str(len(df)))
    df['newfilename'] = (df.index + 1).astype(str).str.zfill(seqlen) + '.' +df.ext

    #if not data:
     #   return
    
    # track the seq
    i = 1
    
    # keep the track of the new file names in a list in order to be used later
    tmp_file_names = []
    
    # first rename the files with _tmp in order not to overwrite files 
    # and keep the renamed files in the list 
    # then use the list to remove _tmp files
    for index, row in df.iterrows():
        tmp_file_name = path+'/'+row['newfilename'] + '_tmp'
        orig_file_name =  path+'/'+row['label']+'_'+row['file'] +'.'+ row['ext']
        print('rename %s to %s'  % (orig_file_name, tmp_file_name) )
        tmp_file_names.append(tmp_file_name)

        
    # now remove _tmp files    
    for temp_file in tmp_file_names:
        print('rename %s to %s'  % (temp_file, temp_file.rsplit('_tmp', -1)[0]) )
        #os.rename(temp_file, temp_file.rsplit('_tmp', -1)[0]) 
    return df

In [231]:
df =shuffle_files_test_dataset(path)

rename /ds/dataset/quran/112/112_0426.mp3 to /ds/dataset/quran/112/001.mp3_tmp
rename /ds/dataset/quran/112/112_0194.mp3 to /ds/dataset/quran/112/002.mp3_tmp
rename /ds/dataset/quran/112/112_0398.mp3 to /ds/dataset/quran/112/003.mp3_tmp
rename /ds/dataset/quran/112/112_0181.mp3 to /ds/dataset/quran/112/004.mp3_tmp
rename /ds/dataset/quran/112/112_0072.mp3 to /ds/dataset/quran/112/005.mp3_tmp
rename /ds/dataset/quran/112/112_0016.mp3 to /ds/dataset/quran/112/006.mp3_tmp
rename /ds/dataset/quran/112/112_0179.mp3 to /ds/dataset/quran/112/007.mp3_tmp
rename /ds/dataset/quran/112/112_0454.mp3 to /ds/dataset/quran/112/008.mp3_tmp
rename /ds/dataset/quran/112/112_0386.mp3 to /ds/dataset/quran/112/009.mp3_tmp
rename /ds/dataset/quran/112/112_0066.mp3 to /ds/dataset/quran/112/010.mp3_tmp
rename /ds/dataset/quran/112/112_0328.mp3 to /ds/dataset/quran/112/011.mp3_tmp
rename /ds/dataset/quran/112/112_0023.mp3 to /ds/dataset/quran/112/012.mp3_tmp
rename /ds/dataset/quran/112/112_0176.mp3 to /ds/dat